In [233]:
# ignores warnings 
import warnings
warnings.filterwarnings('ignore')

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import tarfile
from scipy import stats
from api_key import api_key



In [234]:
# set the year 
# since=2005
# until=2016

In [235]:
# pull the API
# url_crime=(f"https://api.usa.gov/crime/fbi/sapi/api/data/arrest/national/offense/monthly/{since}/{until}?API_KEY=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv")
# url_crime

In [236]:
# Make pretty print
#response = requests.get(url_crime)
#response_json = response.json()
#print(json.dumps(response_json, indent=4, sort_keys=True))

In [237]:
## fine out the size of the tar file
tar = tarfile.open("Crime_DataFiles/DATA_01-2013_03-2018.tar.gz", "r:gz")
for tarinfo in tar:
    print(tarinfo.name, "is", tarinfo.size, "bytes in size and is ", end="")
    if tarinfo.isreg():
        print("a regular file.")
    elif tarinfo.isdir():
        print("a directory.")
    else:
        print("something else.")
tar.close()

stage3.csv is 149698584 bytes in size and is a regular file.


In [238]:
# read tar g.zip file - tar.zip file can load to main repo 
df = pd.read_csv('Crime_DataFiles/DATA_01-2013_03-2018.tar.gz', compression='gzip', header=0, sep=',', error_bad_lines=False)


In [254]:
# check column values
crime_df = pd.DataFrame(df)
print(crime_df.columns.values)



['stage3.csv' 'date' 'state' 'city_or_county' 'address' 'n_killed'
 'n_injured' 'incident_url' 'source_url' 'incident_url_fields_missing'
 'congressional_district' 'gun_stolen' 'gun_type'
 'incident_characteristics' 'latitude' 'location_description' 'longitude'
 'n_guns_involved' 'notes' 'participant_age' 'participant_age_group'
 'participant_gender' 'participant_name' 'participant_relationship'
 'participant_status' 'participant_type' 'sources' 'state_house_district'
 'state_senate_district']


In [256]:
#purpose of viewing data only
#crime_df['participant_age_group'].unique()


In [242]:
# create crime dataframe
crime_df = pd.DataFrame(df, columns = ['stage3.csv','date', 'state', 'city_or_county', 'n_killed', 'n_injured', 'participant_gender','participant_age_group','latitude','longitude'])
crime_df.rename(columns ={'stage3.csv':'Incident','date':'Date','state':'State','city_or_county':'City or Country',
                           'n_killed':'Number Killed','n_injured':'Number Injured','participant_gender':'Gender Involved',
                           'participant_age_group':'Age Group','latitude':'Latitude','longitude':'Longitude'}, inplace=True)
crime_df.head()

,Incident,Date,State,City or Country,Number Killed,Number Injured,Gender Involved,Age Group,Latitude,Longitude
0,461105.0,2013-01-01,Pennsylvania,Mckeesport,0.0,4.0,0::Male||1::Male||3::Male||4::Female,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,40.3467,-79.8559
1,460726.0,2013-01-01,California,Hawthorne,1.0,3.0,0::Male,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,33.9090,-118.3330
2,478855.0,2013-01-01,Ohio,Lorain,1.0,3.0,0::Male||1::Male||2::Male||3::Male||4::Male,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,41.4455,-82.1377
3,478925.0,2013-01-05,Colorado,Aurora,4.0,0.0,0::Female||1::Male||2::Male||3::Male,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,39.6518,-104.8020
4,478959.0,2013-01-07,North Carolina,Greensboro,2.0,2.0,0::Female||1::Male||2::Male||3::Female,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,36.1140,-79.9569


In [243]:
# Get only the 1.1 year of data for Obama and Trump Presidency
ObamaTrump_df = crime_df.loc[(crime_df['Date']>='2015-01-01') & (crime_df['Date']<'20190101')]
#len(ObamaTrump_df)
#ObamaTrump_df['Male']=''
#ObamaTrump_df['Female']=''
#ObamaTrump_df

In [221]:
#ObamaTrump_df.dropna(inplace=True)

In [244]:
# Add Column Name - Presidency and fill in base on 2 years Obama and Trump was President
ObamaTrump_df.loc[(ObamaTrump_df['Date']>='2015-01-01') & (ObamaTrump_df['Date'] <'2016-04-01'), 'Presidency'] ='Obama'
ObamaTrump_df.loc[(ObamaTrump_df['Date']>='2017-01-01') & (ObamaTrump_df['Date'] <'2018-04-01'), 'Presidency'] ='Trump'
ObamaTrump_df

,Incident,Date,State,City or Country,Number Killed,Number Injured,Gender Involved,Age Group,Latitude,Longitude,Presidency
52132,274168.0,2015-01-01,Oklahoma,Tulsa,0.0,2.0,0::Female||1::Male,0::Adult 18+||1::Adult 18+,36.0934,-95.8870,Obama
52133,276211.0,2015-01-01,Louisiana,Labadieville,1.0,0.0,0::Male,0::Adult 18+,29.8312,-90.9609,Obama
52134,272302.0,2015-01-01,Mississippi,Hattiesburg,0.0,0.0,0::Male,0::Adult 18+,31.3271,-89.2903,Obama
52135,272482.0,2015-01-01,Alabama,Bessemer,0.0,2.0,0::Male||1::Male,NaN,33.4423,-86.9322,Obama
52136,272487.0,2015-01-01,Illinois,Chicago,0.0,1.0,0::Male,NaN,41.9202,-87.7857,Obama
...,...,...,...,...,...,...,...,...,...,...,...
239672,1083142.0,2018-03-31,Louisiana,Rayne,0.0,0.0,0::Female,0::Adult 18+,NaN,NaN,Trump
239673,1083139.0,2018-03-31,Louisiana,Natchitoches,1.0,0.0,0::Male||1::Male,0::Adult 18+||1::Adult 18+,31.7537,-93.0836,Trump
239674,1083151.0,2018-03-31,Louisiana,Gretna,0.0,1.0,0::Male,0::Adult 18+,29.9239,-90.0442,Trump
239675,1082514.0,2018-03-31,Texas,Houston,1.0,0.0,0::Male,0::Adult 18+,29.7201,-95.6110,Trump


In [247]:
ObamaTrump_df['Male'] = ObamaTrump_df['Gender Involved'].str.count('Male')
ObamaTrump_df['Female'] = ObamaTrump_df['Gender Involved'].str.count('Female')
ObamaTrump_df['Child 0-11'] = ObamaTrump_df['Age Group'].str.count('Child 0-11')
ObamaTrump_df['Teen 12-17'] = ObamaTrump_df['Age Group'].str.count('Teen 12-17')
ObamaTrump_df['Adult 18+'] = ObamaTrump_df['Age Group'].str.count('Adult 18+')
ObamaTrump_df['Year Month'] = [''.join(x.split('-')[0:2]) for x in ObamaTrump_df.Date]
ObamaTrump_df.head()

,Incident,Date,State,City or Country,Number Killed,Number Injured,Gender Involved,Age Group,Latitude,Longitude,Presidency,Male,Female,Child 0-11,Teen 12-17,Adult 18+,Year Month
52132,274168.0,2015-01-01,Oklahoma,Tulsa,0.0,2.0,0::Female||1::Male,0::Adult 18+||1::Adult 18+,36.0934,-95.8870,Obama,1.0,1.0,0.0,0.0,2.0,201501
52133,276211.0,2015-01-01,Louisiana,Labadieville,1.0,0.0,0::Male,0::Adult 18+,29.8312,-90.9609,Obama,1.0,0.0,0.0,0.0,1.0,201501
52134,272302.0,2015-01-01,Mississippi,Hattiesburg,0.0,0.0,0::Male,0::Adult 18+,31.3271,-89.2903,Obama,1.0,0.0,0.0,0.0,1.0,201501
52135,272482.0,2015-01-01,Alabama,Bessemer,0.0,2.0,0::Male||1::Male,NaN,33.4423,-86.9322,Obama,2.0,0.0,NaN,NaN,NaN,201501
52136,272487.0,2015-01-01,Illinois,Chicago,0.0,1.0,0::Male,NaN,41.9202,-87.7857,Obama,1.0,0.0,NaN,NaN,NaN,201501


In [249]:
# check validate data
output_file = "Crime_DataFiles/cime_analysis.csv"
ObamaTrump_df.to_csv(output_file, index = False)

In [225]:
# Count incident by Presidency and graph
CrimeByPres_df = ObamaTrump_df.groupby('Presidency')['Incident'].count()
CrimeByPres_df

Presidency
Obama    67028
Trump    75203
Name: Incident, dtype: int64

In [200]:
CrimeBySt_df = ObamaTrump_df.groupby('State')['Incident'].count().nlargest(10)
CrimeBySt_df

State
Illinois          14443
California        12535
Florida           11878
Texas             10433
Ohio               7866
New York           7793
Georgia            6887
Pennsylvania       6642
North Carolina     6562
Louisiana          6188
Name: Incident, dtype: int64